In [10]:

from sqlalchemy import create_engine
import numpy as np
import pandas as pd
import logging
import os
import cv2




In [11]:
root_folder='D:\\'
image_dir = os.path.join(root_folder,'new')
dataframes_folder=os.path.join(root_folder,'dataframes')
images_binary_folder=os.path.join(root_folder,'images_binary')
if not os.path.exists(dataframes_folder):
    os.mkdir(dataframes_folder)
if not os.path.exists(images_binary_folder):
    os.mkdir(images_binary_folder)
existing_item_ids=[os.path.splitext(filename)[0] for filename in os.listdir(dataframes_folder)]

In [12]:
# Configure logging format and level (optional)
logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO  # Set the logging level (INFO, DEBUG, WARNING, ERROR, CRITICAL)
)

# Initialize the logger
logger = logging.getLogger('my_logger')

# Define the file handler to log messages to a file
log_file = 'app.log'
file_handler = logging.FileHandler(log_file)

# Set the logging format for the file handler
file_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(file_formatter)

# Add the file handler to the logger
logger.addHandler(file_handler)

In [13]:
image_size = (224, 224)
def load_and_preprocess_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, image_size)
    img = img.astype('float32') / 255.0  # Normalize pixel values
    return img


In [22]:
def process_images_save_dataframe(item_id, item_df, image_dir):
    binary_item_path = os.path.join(images_binary_folder, str(item_id))
    csv_path = os.path.join(dataframes_folder, f'{item_id}.csv')

    item_folder = os.path.join(image_dir, str(item_id))
    
    if os.path.exists(item_folder):
        img_files = [f for f in os.listdir(item_folder) if os.path.isfile(os.path.join(item_folder, f))
                     and f.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif'))]
        
        images = []
        for img_file in img_files:
            img_path = os.path.join(item_folder, img_file)
            try:
                img_array = load_and_preprocess_image(img_path)
                npy_filename = os.path.splitext(img_file)[0] + '.npy'
                
                if not os.path.exists(binary_item_path):
                    os.mkdir(binary_item_path)
                    
                npy_path = os.path.join(binary_item_path, npy_filename)
                np.save(npy_path, img_array)
                images.append(npy_path)
            except Exception as e:
                logger.error(e)
        
        dfs_list = [pd.concat([pd.DataFrame(row).T]*len(images), ignore_index=True) for _, row in item_df.iterrows()]
        
        for df, img_path in zip(dfs_list, images * len(item_df)):
            df['ImagePath'] = img_path
        
        final_df = pd.concat(dfs_list, ignore_index=True)
        final_df.to_csv(csv_path)

In [15]:
connection_string = (
    "mssql+pyodbc://@DESKTOP-3N7733F\\SQLEXPRESS/FashionFlow?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server"
)
engine = create_engine(connection_string)

In [16]:
sql_query = """
SELECT * FROM ViewFullItem
"""

# Execute the query and load data into a DataFrame
df = pd.read_sql_query(sql_query, engine)
df['ImagePath'] = None
dfs_dict = dict(tuple(df.groupby('ItemId')))

55482


In [21]:
directories=os.listdir(images_binary_folder)
items = [item for item in dfs_dict.items() if str(item[0]) not in directories]
print(len(items))
for item_id, item_df in items:
    process_images_save_dataframe(item_id, item_df, image_dir)


KeyboardInterrupt: 